In [1]:
import pandas as pd
import numpy as np

# Data Preprocessing

In [2]:
df = pd.read_csv("Edge-IIoTset/DNN-EdgeIIoT-dataset.csv", low_memory=True)

/tmp/ipykernel_5413/2263545714.py:1: DtypeWarning: Columns (2,3,6,11,13,14,15,16,17,31,32,34,39,45,51,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Edge-IIoTset/DNN-EdgeIIoT-dataset.csv", low_memory=True)


In [3]:
df.head()

,frame.time,ip.src_host,ip.dst_host,arp.dst.proto_ipv4,arp.opcode,arp.hw.size,arp.src.proto_ipv4,icmp.checksum,icmp.seq_le,icmp.transmit_timestamp,...,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,Attack_label,Attack_type
0,2021 11:44:10.081753000,192.168.0.128,192.168.0.101,0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,Normal
1,2021 11:44:10.162218000,192.168.0.101,192.168.0.128,0,0.0,0.0,0,0.0,0.0,0.0,...,4.0,MQTT,0,0.0,4.0,0.0,0.0,0.0,0,Normal
2,2021 11:44:10.162271000,192.168.0.128,192.168.0.101,0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,Normal
3,2021 11:44:10.162641000,192.168.0.128,192.168.0.101,0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,Normal
4,2021 11:44:10.166132000,192.168.0.101,192.168.0.128,0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,Temperature_and_Humidity,24.0,0.0,0.0,0.0,0.0,0,Normal


In [4]:
from sklearn.utils import shuffle
drop_columns = ["frame.time", "ip.src_host", "ip.dst_host", "arp.src.proto_ipv4","arp.dst.proto_ipv4", 

         "http.file_data","http.request.full_uri","icmp.transmit_timestamp",

         "http.request.uri.query", "tcp.options","tcp.payload","tcp.srcport",

         "tcp.dstport", "udp.port", "mqtt.msg"]

df.drop(drop_columns, axis=1, inplace=True)

df.dropna(axis=0, how='any', inplace=True)

df.drop_duplicates(subset=None, keep="first", inplace=True)

df = shuffle(df)

df.isna().sum()

arp.opcode                   0
arp.hw.size                  0
icmp.checksum                0
icmp.seq_le                  0
icmp.unused                  0
http.content_length          0
http.request.method          0
http.referer                 0
http.request.version         0
http.response                0
http.tls_port                0
tcp.ack                      0
tcp.ack_raw                  0
tcp.checksum                 0
tcp.connection.fin           0
tcp.connection.rst           0
tcp.connection.syn           0
tcp.connection.synack        0
tcp.flags                    0
tcp.flags.ack                0
tcp.len                      0
tcp.seq                      0
udp.stream                   0
udp.time_delta               0
dns.qry.name                 0
dns.qry.name.len             0
dns.qry.qu                   0
dns.qry.type                 0
dns.retransmission           0
dns.retransmit_request       0
dns.retransmit_request_in    0
mqtt.conack.flags            0
mqtt.con

In [5]:
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn import preprocessing

def encode_text_dummy(df, name):

    dummies = pd.get_dummies(df[name])

    for x in dummies.columns:

        dummy_name = f"{name}-{x}"

        df[dummy_name] = dummies[x]
        
        df[dummy_name] *= 1

    df.drop(name, axis=1, inplace=True)

encode_text_dummy(df,'http.request.method')

encode_text_dummy(df,'http.referer')

encode_text_dummy(df,"http.request.version")

encode_text_dummy(df,"dns.qry.name.len")

encode_text_dummy(df,"mqtt.conack.flags")

encode_text_dummy(df,"mqtt.protoname")

encode_text_dummy(df,"mqtt.topic")


In [9]:
X = df.drop(columns=['Attack_type', 'Attack_label'])
# y_bin = df[['Attack_label']]
y_mul = df[['Attack_type']]
# y_mul_group
del df

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_mul, test_size=0.3)
del X, y_mul

In [9]:
obj_exists = []
for col in X_train.columns:
  if X_train[col].dtype == object:
    obj_exists.append(col)
print(len(obj_exists))

0


In [11]:
columns = X_train.columns
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
y_train.value_counts()

Attack_type          
Normal                   954924
DDoS_UDP                  84885
DDoS_ICMP                 47384
SQL_injection             35567
Vulnerability_scanner     35104
DDoS_TCP                  34981
Password                  34973
DDoS_HTTP                 33849
Uploading                 25859
Backdoor                  16901
Port_Scanning             14070
XSS                       10607
Ransomware                 6804
Fingerprinting              595
MITM                        266
Name: count, dtype: int64

In [12]:
from imblearn.over_sampling import SMOTE

minority_classes = [
    # 'Port_Scanning',
    # 'XSS',
    'Ransomware',
    'Fingerprinting',
    'MITM']

desired_samples = {
    # 'Port_Scanning': 20000,
    # 'XSS': 20000,
    'Ransomware': 15000,
    'Fingerprinting': 15000,
    'MITM': 15000
}

smote = SMOTE(sampling_strategy=desired_samples)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [12]:
X_train = pd.DataFrame(X_train, columns=columns)
X_test = pd.DataFrame(X_test, columns=columns)

In [13]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
ohe.fit(y_train)
y_train = pd.DataFrame(
    ohe.transform(y_train).toarray(),
    columns=ohe.categories_
)
y_test = pd.DataFrame(
    ohe.transform(y_test).toarray(),
    columns=ohe.categories_
)
y_train = y_train.astype('int8')
y_test = y_test.astype('int8')

In [ ]:
X_train.to_csv("Edge-IIoTset/X_train.csv", index_label=False)
X_test.to_csv("Edge-IIoTset/X_test.csv", index_label=False)
y_train.to_csv("Edge-IIoTset/y_train.csv", index_label=False)
y_test.to_csv("Edge-IIoTset.y_test.csv", index_label=False)

# DNN Model

In [14]:
import tensorflow as tf
n_classes = 15
layers = [
  tf.keras.layers.Dense(X_train.shape[0], activation='relu', input_shape=(X_train.shape[1],)),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(n_classes, activation='softmax')
]

2023-07-11 15:12:34.422288: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-11 15:12:36.345165: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-11 15:12:36.347404: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-11 15:12:42.168516: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
dnn = tf.keras.Sequential(layers)
dnn.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy',
               tf.keras.metrics.Precision(),
               tf.keras.metrics.Recall()
              ]
    )
dnn.fit(x=X_train, y=y_train.values, epochs=5, batch_size=1024)